In [1]:
import quimb.tensor as qtn
import torch
import symmray as sr
import numpy as np
from functools import partial

# torch.set_default_device("cuda:0") # GPU
torch.set_default_device("cpu") # CPU

Lx = 4
Ly = 2
nsites = Lx * Ly
N_f = nsites  # half-filling
D = 4
chi = -1
seed = 0
# only the flat backend is compatible with jax.jit
flat = False

# generate random binary string with Nf ones and Lx*Ly-Nf zeros
def generate_initial_config(Nf, Lx, Ly):
    np.random.seed(seed)
    total_sites = Lx * Ly
    config = np.array([1] * Nf + [0] * (total_sites - Nf))
    np.random.shuffle(config)
    return config

rcharge_config = generate_initial_config(N_f, Lx, Ly)

def site_charge(site, sites):
    charge_dict = dict(zip(sites, rcharge_config))
    return charge_dict[site]

peps = sr.networks.PEPS_fermionic_rand(
    "U1",
    Lx,
    Ly,
    D,
    phys_dim=4,
    # phys_dim=[
    #     (0, 0),  # linear index 0 -> charge 0, offset 0
    #     (1, 0),  # linear index 1 -> charge 1, offset 0
    #     (1, 1),  # linear index 2 -> charge 1, offset 1
    #     (0, 1),  # linear index 3 -> charge 0, offset 1
    # ],  # -> (0, 3), (2, 1)
    site_charge=partial(site_charge, sites=[(x, y) for x in range(Lx) for y in range(Ly)]),
    subsizes="equal",
    flat=flat,
    seed=seed,
    dtype="float64"
)
peps.tensors[0].data._dummy_modes

((0, 0)-,)

In [46]:
from symmray import FermionicOperator
from autoray import do
import symmray as sr

# Define the symmray amplitude function
def amplitude(fx, peps):
    # convert neighboring pairs (up, down) to single index 0..3
    # these should match up with the phys_dim ordering above
    if fx.shape[0] == 2 * peps.nsites:
        fx = fx[::2] + 2*fx[1::2] # grouped by sites turned into tn indices

    selector = {peps.site_ind(site): val for site, val in zip(peps.sites, fx)}
    tnb = peps.isel(selector)
    return tnb.contract()

# Benchmark amplitude function from Sijing
def get_amp(peps, config, deep=False):
    """Slicing to get the amplitude, faster than contraction with a tensor product state."""
    if deep:
        import copy
        peps = copy.deepcopy(peps)
    else:
        peps = peps.copy()
    if peps.arrays[0].symmetry == 'Z2':
        index_map = {0: 0, 1: 1, 2: 1, 3: 0}
        array_map = {
            0: do('array', [1.0, 0.0]),
            1: do('array', [1.0, 0.0]),
            2: do('array', [0.0, 1.0]),
            3: do('array', [0.0, 1.0])
        }

    for n, site in zip(config, peps.sites):
        p_ind = peps.site_ind_id.format(*site)
        site_id = peps.sites.index(site)
        site_tag = peps.site_tag_id.format(*site)
        # fts = peps.tensors[site_id]
        fts = peps[site_tag]
        ftsdata = fts.data # this is the on-site fermionic tensor (f-tensor) to be contracted
        ftsdata.phase_sync(inplace=True) # explicitly apply all lazy phases that are stored and not yet applied
        phys_ind_order = fts.inds.index(p_ind)
        charge = index_map[int(n)] # charge of the on-site fermion configuration
        input_vec = array_map[int(n)] # input vector of the on-site fermion configuration
        charge_sec_data_dict = ftsdata.blocks # the dictionary of the f-tensor data

        new_fts_inds = fts.inds[:phys_ind_order] + fts.inds[phys_ind_order + 1:] # calculate indices of the contracted f-tensor
        new_charge_sec_data_dict = {} # new dictionary to store the data of the contracted f-tensor
        for charge_blk, data in charge_sec_data_dict.items():
            if charge_blk[phys_ind_order] == charge:
                # 1. Determine which index to select (0 or 1) from the input vector.
                #    `argmax` finds the position of the '1.0'.
                # select_index = torch.argmax(input_vec).item()
                select_index = do('argmax', input_vec)

                # 2. Build the slicer tuple dynamically.
                #    This creates a list of `slice(None)` (which is equivalent to `:`)
                #    and inserts the `select_index` at the correct position.
                slicer = [slice(None)] * data.ndim
                slicer[phys_ind_order] = select_index

                # 3. Apply the slice to get the new data.
                new_data = data[tuple(slicer)]

                # 4. Fermionic sign correction due to potential permutation of odd indices.
                #     (In our convention the physical ind should be the last ind during contraction)
                if charge % 2 != 0 and phys_ind_order != len(charge_blk) - 1:
                    # Count how many odd indices are to the right of the physical index.
                    # Check if odd physical ind permutes through odd number of odd indices.
                    num_odd_right_blk = sum(1 for i in charge_blk[phys_ind_order + 1:] if i % 2 == 1)
                    if num_odd_right_blk % 2 == 1:
                        # local fermionic sign change due to odd parity inds + odd permutation
                        new_data = -new_data

                new_charge_blk = charge_blk[:phys_ind_order] + charge_blk[phys_ind_order + 1:] # new charge block
                new_charge_sec_data_dict[new_charge_blk] = new_data # new charge block and its corresponding data in a dictionary

        new_duals = ftsdata.duals[:phys_ind_order] + ftsdata.duals[phys_ind_order + 1:]

        if int(n) == 1:
            new_dummy_modes = (3 * site_id + 1) * (-1)
        elif int(n) == 2:
            new_dummy_modes = (3 * site_id + 2) * (-1)
        elif int(n) == 3 or int(n) == 0:
            new_dummy_modes = ()
        
        new_dummy_modes1 = FermionicOperator(new_dummy_modes, dual=True) if new_dummy_modes else ()
        new_dummy_modes = ftsdata.dummy_modes + (new_dummy_modes1,) if isinstance(new_dummy_modes1, FermionicOperator) else ftsdata.dummy_modes
        dummy_modes = list(new_dummy_modes)[::-1]
        try:
            if peps.arrays[0].symmetry == 'Z2':
                new_charge = (charge + ftsdata.charge) % 2 # Z2 symmetry, charge should be 0 or 1
            new_fts_data = sr.FermionicArray.from_blocks(new_charge_sec_data_dict, duals=new_duals, charge=new_charge, symmetry=peps.arrays[0].symmetry, dummy_modes=dummy_modes)
        except Exception:
            raise ValueError("Error when constructing the new f-tensor after contraction.")
        
        fts.modify(data=new_fts_data, inds=new_fts_inds, left_inds=None)

    amp = qtn.PEPS(peps)

    return amp

fx = torch.tensor([1, 2, 1, 2, 1, 2, 1, 2])

amplitude(fx, peps)

np.float64(27.6452982872981)

In [47]:
# Convert U(1) PEPS to Z2 PEPS
def u1arr_to_z2arr(u1array):
    """
    Convert a FermionicArray with U1 symmetry to a FermionicArray with Z2 symmetry
    """
    def u1ind_to_z2indmap(u1indices):
        index_maps = []
        for blockind in u1indices:
            index_map = {}
            indicator = 0 #max value=blocind.size_total-1
            for c, dim in blockind.chargemap.items():
                for i in range(indicator, indicator+dim):
                    index_map[i]=int(c%2)
                indicator+=dim
            index_maps.append(index_map)
        return index_maps
    
    u1indices = u1array.indices
    u1charge = u1array.charge
    u1dummy_modes = u1array.dummy_modes
    u1duals = u1array.duals
    index_maps = u1ind_to_z2indmap(u1indices)
    z2array=sr.Z2FermionicArray.from_dense(u1array.to_dense(), index_maps=index_maps, duals=u1duals, charge=u1charge%2, dummy_modes=u1dummy_modes)
    return z2array

def u1peps_to_z2peps(peps):
    """
    Convert a PEPS with U1 symmetry to a PEPS with Z2 symmetry
    """
    pepsu1 = peps.copy()
    for ts in pepsu1.tensors:
        ts.modify(data=u1arr_to_z2arr(ts.data))
    return pepsu1.copy()

z2peps = u1peps_to_z2peps(peps)
for site in z2peps.sites:
    z2peps[site].data._label = site
for ts in z2peps.tensors:
    ts.data.indices[-1]._linearmap = ((0, 0), (1, 0), (1, 1), (0, 1))
# Compare the two amplitude functions
get_amp(z2peps, fx).contract(), amplitude(fx, z2peps)

(np.float64(27.645298287298107), np.float64(27.645298287298107))